In [27]:
import os
import re
import sys
import csv
import time
import tqdm
import nltk
import math
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import tree
from sklearn import datasets
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.corpus import wordnet
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import normalize
from sklearn.feature_selection import chi2
from scipy.stats import uniform
from scipy.stats import randint
from spellchecker import SpellChecker #need to install for some computers
from Test import * 
from BerNB import *


feature_names = []

vectorizer = TfidfVectorizer(ngram_range=(1,1),norm='l2',sublinear_tf=False)
#vectorizer = CountVectorizer(ngram_range=(1,1),stop_words='english',binary=True)

corpus_train = pd.read_csv("reddit_train.csv",usecols=['comments','subreddits'],delimiter=',')
corpus_test = pd.read_csv("reddit_test.csv",usecols=['comments'],delimiter=',')
english_words = set(nltk.corpus.words.words()\
                    + nltk.corpus.gutenberg.words()\
                    + nltk.corpus.webtext.words()\
                    + nltk.corpus.nps_chat.words()\
                    + nltk.corpus.brown.words())
#                     + nltk.corpus.reuters.words()\

#spell = SpellChecker()

# def is_english_word(x):
#     return (spell.correction(x) in english_words)

# important for cleaning non-english tokens
#spell = SpellChecker()
# delete_list = ["aaa" ,"bbb","ccc","ddd","eee","fff","ggg","hhh","iii","jjj","kkk","lll","mmm","nnn","ooo","ppp","qqq","rrr","sss","ttt"\
#                ,"uuu","vvv","www","xxx","yyy","zzz"]

def misspell(x):
    if "aaa" not in x and "bbb" not in x and "ccc" not in x and "ddd" not in x \
    and "eee" not in x and "fff" not in x and "ggg" not in x and "hhh" not in x \
    and "iii" not in x and "jjj" not in x and "kkk" not in x and "lll" not in x and "mmm" not in x and "nnn" not in x \
    and "ooo" not in x and "ppp" not in x and "qqq" not in x and "rrr" not in x and "sss" not in x and "ttt" not in x \
    and "uuu" not in x and "vvv" not in x and "www" not in x and "xxx" not in x and "yyy" not in x and "zzz" not in x \
    and "aa" not in x and "zz" not in x:
        return True
    else:
        return False
    

# delete_list = ["aaa" not in x,"bbb" not in x,"ccc" not in x,"ddd" not in x,"eee" not in x,"fff" not in x,"ggg" not in x,\
#                "hhh" not in x,"iii" not in x,"jjj" not in x,"kkk" not in x,"lll" not in x,"mmm" not in x,"nnn" not in x,\
#                "ooo" not in x,"ppp" not in x,"qqq" not in x,"rrr" not in x,"sss" not in x,"ttt" not in x
#                ,"uuu" not in x,"vvv" not in x,"www" not in x,"xxx" not in x,"yyy" not in x,"zzz" not in x]

# a helper function to process one comment
def preprocess_text(text): 
    text = text.lower().split()
    stops = set(stopwords.words("english"))
#     text = [w for w in text if not w in stops and len(w) >= 3]
    text = " ".join(text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"^https?:\/\/.*[\r\n]*","", text)
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ',text)
    text = text.split()
    
#     # lemmatization
#     lemma = nltk.wordnet.WordNetLemmatizer()
#     text = [lemma.lemmatize(word) for word in text]
    
#     # stemming
#     text = [PorterStemmer().stem(word) for word in text]
    
    text_final = []
    
    # clean all non-English words, numbers, and other weirdos, stopwords
    for x in text:
        #x = spell.correction(x)
        if x in english_words: # and len(x)<20 and len(x) > 2 and misspell(x):# and is_english_word(x):
            text_final.append(x)
    
    text = " ".join(text_final)
    return text


# the major function to process the training dataset
# returns (1) a matrix of all training features x (2) a numpy array of y labels
def preprocess():
    df = corpus_train.copy()
    df['comments'] = df['comments'].map(lambda x: preprocess_text(x))
    y_train = df["subreddits"].to_numpy()
    x_train = vectorizer.fit_transform(df['comments'])
    
    print(vectorizer.get_feature_names())
    global feature_names
    feature_names = vectorizer.get_feature_names()
#     featname=vectorizer.get_feature_names()
#     chi_squared,pval = chi2(x_train, y_train)
#     featname = pd.DataFrame(featname)
#     chi_2 = pd.DataFrame(chi_squared)
#     pval = pd.DataFrame(pval)

#     data = pd.concat([featname,chi_2,pval],axis=1)
#     data.columns = ["word","chi_squared","pval"]
#     data = data.sort_values("pval",axis=0)

#     to_delete = list(data.index[10000:])     # 保留10000-15000之间最高
#     all_cols = np.arange(x_train.shape[1])
#     cols_to_keep = np.where(np.logical_not(np.in1d(all_cols, to_delete)))[0]
#     x_train = x_train[:, cols_to_keep]
    return x_train, y_train


# function to process the testing set
# returns the matrix of all testing features
def preprocess_testing():
    df = corpus_test.copy()
    df['comments'] = df['comments'].map(lambda x: preprocess_text(x))
    #y_train = df["subreddits"].to_numpy()
    x_train = vectorizer.fit_transform(df['comments'])
    return x_train
    

In [28]:
x,y = preprocess()

['00', '000', '01', '02', '03', '035', '04', '05', '06', '060', '07', '08', '09', '10', '100', '1000', '10000', '1001', '100mb', '101', '102', '1024', '103', '104', '1040', '105', '106', '1066', '107', '108', '109', '10th', '11', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '11th', '12', '120', '1200', '121', '122', '123', '124', '125', '125th', '126', '127', '128', '129', '1299', '12th', '13', '130', '1300', '131', '132', '133', '134', '135', '1350', '136', '137', '138', '139', '13th', '14', '140', '141', '142', '143', '144', '145', '1450', '1453', '146', '147', '148', '149', '1492', '14th', '15', '150', '1500', '151', '152', '153', '155', '156', '157', '158', '159', '1599', '15th', '16', '160', '1600', '1609', '161', '162', '1622', '1626', '163', '1630', '164', '1640', '1642', '165', '166', '1667', '167', '168', '1680', '1687', '1688', '169', '16th', '17', '170', '1700', '171', '172', '173', '174', '175', '1750', '1751', '1760', '1772', '1773', '1774', '1775'

In [29]:
x.shape

(70000, 31539)

In [ ]:
#######################Stats memo##########################
No. of tokens: training set / test set

No operation: 72990
    
stopwords:72685

lemma: 66658
    
stem: 54040
    
remove-english:31539
    
    
    
    ----------------------
remove stopwords: 62721 / 41331
remove non-English: 20713 / 16111
remove non-Englsih/remove stopwords: 20587 / 15986
    
lemma/remove non-English: 21125 / 16518
stem/remove non-English: 10462 / 8352

    
lemma/remove non-English/remove stopwords: 21002 / 16396
stem/remove non-English/remove stopwords: 10359 / 8250

lemma/stem/remove non-English/remove stopwords: 10323 / 8236


    


In [2]:
x,y = preprocess()
#print(x.shape)

#print(x)



['aba', 'aback', 'abadi', 'abam', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abandonned', 'abandonware', 'abaya', 'abb', 'abbas', 'abbey', 'abbotsford', 'abbreviated', 'abbreviation', 'abby', 'abc', 'abck', 'abd', 'abdelhak', 'abdelkader', 'abdicate', 'abdomen', 'abdominal', 'abducted', 'abducting', 'abduction', 'abductive', 'abdul', 'abdullah', 'abe', 'abeast', 'abed', 'abel', 'abell', 'abendblatt', 'aberation', 'aberdeen', 'aberg', 'aberration', 'abetting', 'abeyta', 'abgelehnt', 'abgrenzung', 'abhor', 'abhorrent', 'abhors', 'abidal', 'abide', 'abiding', 'abilify', 'abilites', 'abilitie', 'ability', 'abillities', 'abilties', 'abin', 'abingdon', 'abington', 'abit', 'abitur', 'abject', 'ablaze', 'able', 'abled', 'ableton', 'abliged', 'abn', 'abnormal', 'abnormality', 'abnormally', 'aboard', 'abode', 'abogado', 'aboiut', 'abolish', 'abolished', 'abolition', 'abolitionist', 'abolve', 'abominably', 'abomination', 'aboot', 'aboriginal', 'aborigine', 'aborigionals', 'abort', 'abo

In [8]:
from sklearn.metrics import accuracy_score
clf = BerNB(alpha=1)
#x,y = preprocess()
clf.fit(x[:int(0.8*x.shape[0])],y[:int(0.8*y.shape[0])])
#print(accuracy_score(clf.predict(x[-1000:]),y[-1000:]))
#model_validate(clf,x,y)



In [9]:
#clf = BerNB(alpha=1)
#x,y = preprocess()
from sklearn.metrics import accuracy_score
accuracy_score(clf.predict(x[int(0.8*x.shape[0]):]),y[int(0.8*x.shape[0]):])

0.5188571428571429

In [10]:
clf = BernoulliNB()
model_validate(clf,x,y)

Result of k-fold cross validation on  BernoulliNB
0.518
